In [1]:
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import tifffile as tiff
import seaborn as sns
import pickle

import networkx as nx
from sklearn.model_selection import train_test_split

In [2]:
import torch_geometric.utils
import torch

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [6]:
import graph
import torch
import torch_geometric.utils
import networkx as nx
import lightning.pytorch as pl
import torch.utils.data as data

spatial_omics_folder = (Path().cwd().parents[0]).absolute() / 'data' / 'spatial_omics_graph_external'
process_path = (Path().cwd().parents[0]).absolute() / 'data' / 'torch_graph_data'

# Create data loader

In [7]:
from torch_geometric.loader import DataLoader
seed = torch.Generator().manual_seed(42)

name = 'Lung_Covid'

# Crate dataset
dataset = graph.GraphDataset(process_path / name, process_path / name / 'info.csv', 4, y_name='label')

train_set, val_set, test_set = graph.train_test_val_split(dataset)

# Create Dataloader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


In [8]:
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.nc}')

Dataset: GraphDataset(237):
Number of graphs: 237
Number of features: 35
Number of classes: 4


In [9]:
print(f'Train set: {len(train_set)}, val set: {len(test_set)}, test set: {len(val_set)}')

Train set: 115, val set: 94, test set: 28


In [10]:
for step, data in enumerate(test_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    data.label
    break

Step 1:
Number of graphs in the current batch: 32
DataBatch(edge_index=[2, 572154], num_nodes=82768, x=[82768, 35], pos=[82768, 2], node_types=[32], label=[32], edge_attr=[572154, 2], y=[32], name=[32], batch=[82768], ptr=[33])



# Train network K Fold

In [11]:
from lightning.pytorch.accelerators import find_usable_cuda_devices
import wandb

In [12]:
from sklearn.model_selection import KFold, StratifiedKFold, ShuffleSplit, StratifiedShuffleSplit
from torch.utils.data import SubsetRandomSampler

In [13]:
condition = 'SF_03172024_Lung'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' / "saved_models" / f"Graph_GNNs_{condition}" 
project_name = f'{condition}'

In [14]:
AVAIL_GPUS = [0]

# Setting the seed
pl.seed_everything(42)

NUM_LAYERS = 3
HIDDEN_CHANNELS = 64
pools = ['mean', 'max', 'attention', 'attention2']
# pools = ['attention2']
models = ['GAT', 'GINConv']
# models = ['GINConv']
epochs = 100

Global seed set to 42


In [15]:
ys = [data.y.item() for data in dataset]

In [16]:
np.unique(ys, return_counts=True)

(array([0, 1, 2, 3]), array([ 37,  16,  64, 120], dtype=int64))

In [17]:
# Print K fold model number of samples and number of positive cases

k_folds = 5
# kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
kfold = StratifiedShuffleSplit(n_splits=k_folds, test_size=0.3, random_state=0)
val_size = 0.2

for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset, ys)):
    # train_subset = dataset.index_select(train_ids.tolist())
    test_subset = dataset.index_select(test_ids.tolist())
    
    train_ids, valid_ids = train_test_split(train_ids, test_size = val_size, random_state=0)
    train_subset = dataset.index_select(train_ids.tolist())
    val_subset = dataset.index_select(valid_ids.tolist())
    
    for pool in pools:
        for model in models:
            if model == 'GAT':
                batch_size=64
            else:
                batch_size=128
            # Path to the folder where the pretrained models are saved
            CHECKPOINT_PATH = checkpoint_folder / f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_{fold}' / pool
            CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

            # Skip already trained kfold and pool
            checkpoint = CHECKPOINT_PATH / f"GraphLevel{model}" / f"GraphLevel{model}.ckpt" 
            if checkpoint.exists():
                print(checkpoint)
                continue

            # Run training
            run = wandb.init(project=project_name, name=f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_{fold}', 
                            group=f'{model}_{pool}')
            graph.train_graph_classifier_kfold(model, 
                                                 train_subset, 
                                                 val_subset, 
                                                 test_subset,
                                                 dataset, 
                                                 CHECKPOINT_PATH, 
                                                 AVAIL_GPUS, 
                                                 hidden_channels=HIDDEN_CHANNELS, 
                                                 num_layers=NUM_LAYERS, 
                                                 epochs=epochs,
                                                 batch_size=batch_size,
                                                 graph_pooling=pool,
                                                 task='multiclass',
                                                 num_classes = dataset.nc,
                                                pos=True)
            run.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\loggers\wandb.py:397: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
You are using a CUDA device ('NVIDIA RTX A2000 12GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint

{'test_acc': tensor(0.6944), 'test_auc': tensor(0.6730), 'test_f1': tensor(0.6944)}


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_auc,▁
test_f1,▁
train_acc,▁▄▄▄▄▄▄▄▄▄▄▇██▄██▇█▇▇███████████████████
train_auc,▁▄▅▅▅▅▅▅▅▆▅▆▆▇▆▇▇▅▆▄▅▇▇██▇▇▇▇▇▇▇▇▇▇▇██▇█
train_f1,▁▄▄▄▄▄▄▄▄▄▄▇██▄██▇█▇▇███████████████████
train_loss_epoch,▆▆▅▅▅▄▅█▆▆▅▃▃▃▄▂▂▄▃▃▃▁▂▄▃▂▃▁▁▁▄▂▁▂▁▂▁▃▃▂
train_loss_step,▇█▄▇▇▇▇▆▄▄▄▁▄▄▄▃▄▇▄▃▂▅▄▁▄▅▄▃▃▄
trainer/global_step,▁▁▁▁▂▂▂▂▁▃▃▃▃▂▄▄▄▄▄▂▅▅▅▅▂▆▆▆▆▃▇▇▇▇▃▇███▃


Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\loggers\wandb.py:397: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_SF_03172024_Lung\GINConv_3_64_0\mean\GraphLevelGINConv exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name        | T

...

{'test_acc': tensor(0.7917), 'test_auc': tensor(0.9099), 'test_f1': tensor(0.7917)}


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_auc,▁
test_f1,▁
train_acc,▁▃▃▃▃▃▃▃▃▃▆▆▇▇▇▇▆▇▇▆▆▇▇▇▇▇▇▇▇▆▇▇▇▇▅█████
train_auc,▁▅▄▄▄▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████▇████▆█████
train_f1,▁▃▃▃▃▃▃▃▃▃▆▆▇▇▇▇▆▇▇▆▆▇▇▇▇▇▇▇▇▆▇▇▇▇▅█████
train_loss_epoch,█▇█▅█▅█▇▇▆▆▆▃▃▅▃▂▃▃▄▆▂▄▃▃▁▁▂▁▃▅▄▃▂▆▁▂▃▆▂
train_loss_step,▇▆▁█▆▇▆▄▁▄▃▁▄▄▄▃▃▄▄▃▂▄▃▃▃▅▂▂▂▄
trainer/global_step,▁▁▁▁▂▂▂▂▁▃▃▃▃▂▄▄▄▄▄▂▅▅▅▅▂▆▆▆▆▃▇▇▇▇▃▇███▃


Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\loggers\wandb.py:397: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_SF_03172024_Lung\GINConv_3_64_4\attention2\GraphLevelGINConv exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name     

{'test_acc': tensor(0.7222), 'test_auc': tensor(0.8894), 'test_f1': tensor(0.7222)}


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_auc,▁
test_f1,▁
train_acc,▁▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅█▇▇████████▇▇██▇█
train_auc,▁▆▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▄▅▅▆▆▇█▇██▇██▇██▇█████
train_f1,▁▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅█▇▇████████▇▇██▇█
train_loss_epoch,▆▅▅▄▅▇▄▇▅▅▃▇▅▅▄▅▄█▄▅▇▆▃▃▅▄▁▁▄▂▁▃▁▁▃▃▁▄▆▂
train_loss_step,▅▅▃▁▅▅▃▆▆█▄▃▄▂▂▃▅▃▁▂
trainer/global_step,▁▁▁▂▁▁▂▂▂▃▃▃▂▂▄▄▄▄▄▄▃▅▅▅▅▅▆▃▃▆▆▇▇▇▇▄▄███


## MLP

In [18]:
AVAIL_GPUS = [0]

# Setting the seed
pl.seed_everything(42)

NUM_LAYERS = 3
HIDDEN_CHANNELS = 64
pools = ['mean', 'max', 'attention', 'attention2']
models = ['MLP']
epochs = 100

Global seed set to 42


In [19]:
ys = [data.y.item() for data in dataset]

In [20]:
# Print K fold model number of samples and number of positive cases

k_folds = 5
# kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
kfold = StratifiedShuffleSplit(n_splits=k_folds, test_size=0.3, random_state=0)
val_size = 0.2

for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset, ys)):
    # train_subset = dataset.index_select(train_ids.tolist())
    test_subset = dataset.index_select(test_ids.tolist())
    
    train_ids, valid_ids = train_test_split(train_ids, test_size = val_size, random_state=0)
    train_subset = dataset.index_select(train_ids.tolist())
    val_subset = dataset.index_select(valid_ids.tolist())
    
    for pool in pools:
        for model in models:
            if model == 'GAT':
                batch_size=64
            else:
                batch_size=128
            # Path to the folder where the pretrained models are saved
            CHECKPOINT_PATH = checkpoint_folder / f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_{fold}' / pool
            CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

            # Skip already trained kfold and pool
            checkpoint = CHECKPOINT_PATH / f"GraphLevel{model}" / f"GraphLevel{model}.ckpt" 
            if checkpoint.exists():
                print(checkpoint)
                continue

            # Run training
            run = wandb.init(project=project_name, name=f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_{fold}', 
                            group=f'{model}_{pool}')
            graph.train_graph_classifier_kfold(model, 
                                                 train_subset, 
                                                 val_subset, 
                                                 test_subset,
                                                 dataset, 
                                                 CHECKPOINT_PATH, 
                                                 AVAIL_GPUS, 
                                                 hidden_channels=HIDDEN_CHANNELS, 
                                                 num_layers=NUM_LAYERS, 
                                                 epochs=epochs,
                                                 batch_size=batch_size,
                                                 graph_pooling=pool,
                                                 task='multiclass',
                                                 num_classes = dataset.nc,
                                                pos=False)
            run.finish()


Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\loggers\wandb.py:397: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_SF_03172024_Lung\MLP_3_64_0\mean\GraphLevelMLP exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name        | Type     

{'test_acc': tensor(0.7222), 'test_auc': tensor(0.9156), 'test_f1': tensor(0.7222)}


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_auc,▁
test_f1,▁
train_acc,▁▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▂▄▄▆▆▇▆▆▇▇▇▇▇▇▇▇▇▆████
train_auc,▁▅▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▆▆▆▆▇█▇▇███████▇█▇████
train_f1,▁▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▂▄▄▆▆▇▆▆▇▇▇▇▇▇▇▇▇▆████
train_loss_epoch,█▃▃▅▆▅▆▆▃▆██▆▆█▆▆▆█▆▅▄▅▂▂▄▄▂▃▃▁▁▃▂▄▃▂▁▃▁
train_loss_step,▃▆▆▆███▃▆▃▂▃▁▃▁▅▃▃▁▂
trainer/global_step,▁▁▁▂▁▁▂▂▂▃▃▃▂▂▄▄▄▄▄▄▃▅▅▅▅▅▆▃▃▆▆▇▇▇▇▄▄███


Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\loggers\wandb.py:397: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_SF_03172024_Lung\MLP_3_64_0\max\GraphLevelMLP exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name        | Type      

{'test_acc': tensor(0.5139), 'test_auc': tensor(0.5929), 'test_f1': tensor(0.5139)}


...

{'test_acc': tensor(0.5139), 'test_auc': tensor(0.5603), 'test_f1': tensor(0.5139)}


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_auc,▁
test_f1,▁
train_acc,▁███████████████████████████████████████
train_auc,▁▅▇▅▆▆▆▇▆▆▆▇▇▇▆▅▇▇▆▅▇▇▇▅▇█▇▆▆█▄▅▇▆▄▇▆▇▆▆
train_f1,▁███████████████████████████████████████
train_loss_epoch,▆▃▅▃▃▃▃▁▆█▃▃▁▁▃▃▃▆▅▆▆▃▅▃▃▃▃▃▅▃▃▃▃▅▆▅▅▅▅▆
train_loss_step,▄▆▆▆▅▆▃▃▃▃▅▅▆▅▆▅▃▁█▆
trainer/global_step,▁▁▁▂▁▁▂▂▂▃▃▃▂▂▄▄▄▄▄▄▃▅▅▅▅▅▆▃▃▆▆▇▇▇▇▄▄███


Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\loggers\wandb.py:397: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_SF_03172024_Lung\MLP_3_64_4\attention2\GraphLevelMLP exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name        | Typ

{'test_acc': tensor(0.7361), 'test_auc': tensor(0.8647), 'test_f1': tensor(0.7361)}


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_auc,▁
test_f1,▁
train_acc,▁▅▅▂▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▆▇▇█████▇███
train_auc,▁▆▆▇▆▇▇▇▆▇▆▆▆▆▇▆▆▇▆▆▇▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇████
train_f1,▁▅▅▂▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▆▇▇█████▇███
train_loss_epoch,█▆██▃▆▃▆▅▆▆█▅▅█▆▆▃█▅█▆▅▅▆▆▅▆▄▃▄▃▃▃▃▁▇▃▁▄
train_loss_step,▄▁▄▄▁▁▇▇▄▁▁▁▄█▂▂▁█▃▄
trainer/global_step,▁▁▁▂▁▁▂▂▂▃▃▃▂▂▄▄▄▄▄▄▃▅▅▅▅▅▆▃▃▆▆▇▇▇▇▄▄███
